In [13]:
%pwd

'/home/ebi/machinelearning/end_to_end_ml_projects/end_to_end_food_vision'

In [2]:
%cd ../

/home/ebi/machinelearning/end_to_end_ml_projects/end_to_end_food_vision


In [50]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    train_dir: Path
    test_dir: Path
    epochs: int


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

@dataclass(frozen=True)
class DataLoaderConfig:
    root_dir: Path
    train_dir: Path
    test_dir: Path
    image_shape: list
    batch_size: int
    class_mode: str

In [51]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self,
        config_filepath = CONFIG_FILE_PATH, 
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # create_directories([self.config.artifacts_root])
    
    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks

        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir=config.root_dir, 
            tensorboard_root_log_dir=config.tensorboard_root_log_dir, 
            checkpoint_model_filepath=config.checkpoint_model_filepath
        )

        return prepare_callbacks_config
    
    def get_training_config(self):
        training = self.config.training
        params = self.params
        prepare_base_model = self.config.prepare_base_model
        data_loader = self.config.data_loader


        create_directories([training.root_dir])

        training_config = TrainingConfig(
            root_dir=training.root_dir, 
            trained_model_path=training.trained_model_path, 
            updated_base_model_path=prepare_base_model.updated_base_model_path, 
            train_dir=data_loader.train_dir, 
            test_dir=data_loader.test_dir, 
            epochs=params.EPOCHS
        )

        return training_config
    def get_data_loader_config(self) -> DataLoaderConfig:
        config = self.config.data_loader
        params = self.params

        data_loader_config = DataLoaderConfig(
            root_dir=config.root_dir,
            train_dir=config.train_dir, 
            test_dir=config.test_dir, 
            image_shape=params.IMAGE_SHAPE, 
            batch_size=params.BATCH_SIZE, 
            class_mode=params.CLASS_MODE
        )

        return data_loader_config

In [52]:

import time
import os
import tensorflow as tf

class PrepareCallbacks:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir, 
            f"tb_log_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath, 
            save_best_only=True
        )
    def get_tb_ckpt_callbacks(self):
        return[
            self._create_tb_callbacks, 
            self._create_ckpt_callbacks
        ]

In [53]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

class DataLoader:
    def __init__(self, config: DataLoaderConfig):
        self.config = config
    
    def load_data(self):
        self.train_dir = self.config.train_dir
        self.test_dir = self.config.test_dir

    def prepare_data(self):
        train_datagen = ImageDataGenerator(rescale=1/255.)
        test_datagen = ImageDataGenerator(rescale=1/255.)

        train_data_10_percent = train_datagen.flow_from_directory(self.train_dir, 
                                                                  target_size=self.config.image_shape, 
                                                                  batch_size=self.config.batch_size, 
                                                                  class_mode=self.config.class_mode)
        test_data = train_datagen.flow_from_directory(self.test_dir, 
                                                      target_size=self.config.image_shape, 
                                                      batch_size=self.config.batch_size, 
                                                      class_mode=self.config.class_mode)
        return train_data_10_percent, test_data


In [54]:

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    
    
    def training(self, callbacks_list: list, train_data, test_data):
        self.history = self.model.fit(
            train_data,
            epochs=self.config.epochs, 
            steps_per_epoch=len(train_data), 
            validation_data=test_data, 
            validation_steps=int(0.15 * len(test_data)), 
            callbacks=callbacks_list
        )

        self.save_model(
            path=self.config.trained_model_path, 
            model=self.model
        )

In [55]:
from cnnClassifier import logger



try:
    config = ConfigurationManager()

    STAGE_NAME = "Load Data"
    logger.info(f">>>>>>>> stage {STAGE_NAME} started <<<<<<<<<<<<")
    data_loader_config = config.get_data_loader_config()
    data_loader = DataLoader(config=data_loader_config)
    data_loader.load_data()
    train, test = data_loader.prepare_data()
    logger.info(f">>>>>>>> stage {STAGE_NAME} completed <<<<<<<<<<\n\nx============x")

    STAGE_NAME = "Prepare callbacks"
    logger.info(f">>>>>>>> stage {STAGE_NAME} started <<<<<<<<<<<<")
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallbacks(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    logger.info(f">>>>>>>> stage {STAGE_NAME} completed <<<<<<<<<<\n\nx============x")


    STAGE_NAME = "Training"
    logger.info(f">>>>>>>> stage {STAGE_NAME} started <<<<<<<<<<<<")
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.training(
        callbacks_list=callback_list, 
        train_data= train, 
        test_data=test
    )
    logger.info(f">>>>>>>> stage {STAGE_NAME} completed <<<<<<<<<<\n\nx============x")



except Exception as e:
    raise e

[2023-10-10 14:56:58,181: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-10-10 14:56:58,185: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-10 14:56:58,186: INFO: 2011624294: >>>>>>>> stage Load Data started <<<<<<<<<<<<]
Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.
[2023-10-10 14:56:58,366: INFO: 2011624294: >>>>>>>> stage Load Data completed <<<<<<<<<<

x============x]
[2023-10-10 14:56:58,367: INFO: 2011624294: >>>>>>>> stage Prepare callbacks started <<<<<<<<<<<<]
[2023-10-10 14:56:58,368: INFO: 2011624294: >>>>>>>> stage Prepare callbacks completed <<<<<<<<<<

x============x]
[2023-10-10 14:56:58,369: INFO: 2011624294: >>>>>>>> stage Training started <<<<<<<<<<<<]
[2023-10-10 14:56:58,371: INFO: common: created directory at: artifacts/training]
47/47 [==============================] - ETA: 0s - loss: 2.3490 - accuracy: 0.0947

/home/ebi/miniconda3/envs/tf_full/lib/python3.9/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


47/47 [==============================] - 42s 820ms/step - loss: 2.3490 - accuracy: 0.0947 - val_loss: 2.3182 - val_accuracy: 0.1141
[2023-10-10 14:57:44,155: INFO: 2011624294: >>>>>>>> stage Training completed <<<<<<<<<<

x============x]
